In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import numpy as np
from common import *

In [5]:
import argparse
parser = argparse.ArgumentParser(description='Run Diagnosis experiments')
parser.add_argument("--data_dir", type=str, required=True)
parser.add_argument('--display', dest='display', action='store_true')
parser.add_argument("--output_dir", type=str)
parser.add_argument("--mock", dest='mock', action='store_true')

args = parser.parse_args(['--data_dir=.', '--output_dir=outputs/', '--display'])

In [4]:
from dataloaders import readmission_dataset
data = readmission_dataset(args)

INFO - 2019-02-20 16:23:26,393 - Reading Structured data ...
INFO - 2019-02-20 16:23:26,826 - Reading Notes ...
INFO - 2019-02-20 16:23:48,958 - Stratifying ...


In [ ]:
from PatientVec.Experiments.training_exps import get_basic_data

In [ ]:
train_data, dev_data = get_basic_data(data, truncate=100)

In [ ]:
data.generate_bowder(train_data, stop_words=True, norm=None)

In [ ]:
train_data.X = data.get_vec_encoding(train_data, _type='tfidf')
dev_data.X = data.get_vec_encoding(dev_data, _type='tfidf')

In [ ]:
train_data.X = np.array(train_data.X.todense())
dev_data.X = np.array(dev_data.X.todense())

In [ ]:
from PatientVec.Experiments.configs import vector_experiment
config = vector_experiment(data)
config

In [ ]:
from trainer import Trainer, Evaluator
from models.Baseline import ClassificationTrainer as BaseCT

In [ ]:
trainer = Trainer(BaseCT, config, _type=data.metrics_type, display_metrics=args.display)
trainer.train(train_data, dev_data, save_on_metric=data.save_on_metric)

evaluator = Evaluator(BaseCT, trainer.model.dirname, _type=data.metrics_type, display_metrics=args.display)
_ = evaluator.evaluate(dev_data, save_results=True)
print('='*300)

In [25]:
from PatientVec.models.baselines.LR import LR, LDA
from PatientVec.Experiments.hyperparam_exps import get_basic_data

train_data, dev_data = get_basic_data(data, truncate=100)
lr = LR({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 'type' : 'classifier', 'norm' : 'l2'})
lr.train(train_data)
lr.evaluate(dev_data, save_results=True)

Pos Percentage [0.19738451]


INFO - 2019-02-20 16:33:21,730 - Maximum Sentence Length 721825.000000 , 100 percentile length 721825.000000 ... 
INFO - 2019-02-20 16:33:21,748 - Pos Percentage of remaining data ... 
INFO - 2019-02-20 16:33:21,771 - [0.19738451]


Pos Percentage [0.20937082]


INFO - 2019-02-20 16:33:24,836 - Maximum Sentence Length 588265.000000 , 100 percentile length 588265.000000 ... 
INFO - 2019-02-20 16:33:24,840 - Pos Percentage of remaining data ... 
INFO - 2019-02-20 16:33:24,846 - [0.20937082]



Fit BOW Classifier ...
Fit TFIDF Classifier ...



BOW
{'accuracy': 0.6985274431057563, 'roc_auc': 0.7496373816388553, 'pr_auc': 0.47691187909315513}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.789,0.470,0.699,0.630,0.722
precision,0.882,0.372,0.699,0.627,0.775
recall,0.715,0.637,0.699,0.676,0.699
support,5906.000,1564.000,7470.000,7470.000,7470.000


TFIDF
{'accuracy': 0.7111111111111111, 'roc_auc': 0.7528239195824092, 'pr_auc': 0.4841267575799175}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.800,0.478,0.711,0.639,0.733
precision,0.882,0.384,0.711,0.633,0.778
recall,0.732,0.632,0.711,0.682,0.711
support,5906.000,1564.000,7470.000,7470.000,7470.000


In [26]:
lr.print_all_features(n=40)

tube narrowing brown requesting thrombocytopenia jaundice guided encephalopathy vre chronically anasarca j enteric picc coagulopathy cancer duoderm jaundiced metastatic tumor coccyx prednisone amiodarone monday meeting endovascular icteric iv_contrast dialysis thoracentesis ascites subdural admissions paracentesis transplant vd onc prognosis dnr cmo
----------
metastatic dialysis pea cancer brown tears times prednisone palliative coarse coccyx chronically tube picc duoderm tumor 7.31 transtentorial coagulopathy listerine iv_contrast paracentesis ankylosing transplant ascites subdural icteric monday amiodarone meeting jaundiced bmt thoracentesis onc admissions endovascular vd prognosis dnr cmo
----------


AttributeError: 'MultiOutputClassifier' object has no attribute 'estimators_'

In [46]:
words_tf = lr.get_features(classifier=lr.tf_idf_classifier, estimator=0, n=50)

Attention Comparison
====================

In [5]:
from PatientVec.models.Vanilla import ClassificationTrainer as BasicCT
from PatientVec.models.Hierarchical import ClassificationTrainer as HierCT
from PatientVec.trainer import Trainer, Evaluator
from PatientVec.Experiments.modifiable_config_exp import vanilla_configs, attention_configs, hierarchical_configs, structured_configs

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [56]:
lstm_attention_config = attention_configs[2](data, structured=True, args=args)
lstm_attn_eval = Evaluator(BasicCT, get_latest_model(os.path.join('outputs', lstm_attention_config['exp_config']['exp_name'])))
dev_data = data.get_data('dev', structured=True)
dev_data = data.filter_data_length(dev_data, truncate=90)
lstm_outputs = lstm_attn_eval.evaluate(dev_data, save_results=False)
del lstm_attn_eval

INFO - 2019-02-20 16:48:08,905 - instantiating class <class 'PatientVec.models.Model.Model'> from params {'type': 'seq_classifier_with_attention', 'embedder': {'type': 'token_embedder', 'vocab_size': 32246, 'embed_size': 200, 'embedding_file': 'preprocess/Readmission//embedding_matrix.npy'}, 'decoder': {'num_layers': 2, 'hidden_dims': [128, 1], 'activations': ['tanh', 'linear']}, 'predictor': {'type': 'binary'}, 'structured': {'use_structured': True, 'structured_dim': 142}, 'encoder': {'type': 'cnn', 'hidden_size': 64, 'kernel_sizes': [3, 5, 7, 9], 'activation': 'relu'}, 'attention': {'similarity': {'type': 'additive', 'hidden_size': 128}}} and extras {}
INFO - 2019-02-20 16:48:08,907 - type = seq_classifier_with_attention
INFO - 2019-02-20 16:48:08,908 - instantiating class <class 'PatientVec.models.modules.Embedder.Embedder'> from params {'type': 'token_embedder', 'vocab_size': 32246, 'embed_size': 200, 'embedding_file': 'preprocess/Readmission//embedding_matrix.npy'} and extras {}
I

Setting Embedding
Pos Percentage [0.20937082]


INFO - 2019-02-20 16:48:11,845 - Maximum Sentence Length 588265.000000 , 90 percentile length 19174.500000 ... 
INFO - 2019-02-20 16:48:11,850 - Pos Percentage of remaining data ... 
INFO - 2019-02-20 16:48:11,856 - [0.19723338]



{'accuracy': 0.8188308790718429, 'roc_auc': 0.7608705858877524, 'pr_auc': 0.4742785860170263}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.896,0.315,0.819,0.605,0.781
precision,0.833,0.619,0.819,0.726,0.791
recall,0.968,0.211,0.819,0.590,0.819
support,5397.000,1326.000,6723.000,6723.000,6723.000


In [57]:
dev_data = data.get_data('dev', structured=True)
dev_data = data.filter_data_length(dev_data, truncate=90)

Pos Percentage [0.20937082]


INFO - 2019-02-20 16:50:00,000 - Maximum Sentence Length 588265.000000 , 90 percentile length 19174.500000 ... 
INFO - 2019-02-20 16:50:00,005 - Pos Percentage of remaining data ... 
INFO - 2019-02-20 16:50:00,011 - [0.19723338]


In [58]:
from attention_comparison import get_common_attended_words
X_1 = [dev_data.X[i] for i in range(len(dev_data.y)) if dev_data.y[i] == 1]
attn_1 = [lstm_outputs['attentions'][i] for i in range(len(dev_data.y)) if dev_data.y[i] == 1]
common_words = get_common_attended_words(X_1, attn_1, n=10)

In [59]:
words, freqs = list(zip(*common_words.most_common(50)))

In [60]:
words = data.vocab.map_to_words(words)

In [61]:
list(zip(words, freqs))

[(',', 411),
 ('.', 318),
 ('with', 305),
 ('of', 243),
 ('and', 212),
 ('disease', 161),
 ('to', 146),
 ('p', 124),
 ('family', 120),
 ('contraindications', 100),
 ('diagnosis', 100),
 ('meeting', 93),
 ('placement', 93),
 ('for', 89),
 ('ascites', 87),
 ('from', 80),
 ('/', 80),
 ('pleural', 80),
 ('skin', 77),
 ('line', 75),
 ('team', 72),
 ('dnr', 69),
 ('coccyx', 68),
 ('metastatic', 62),
 (':', 61),
 ('duoderm', 59),
 ('edema', 57),
 ('multiple', 56),
 ('care', 55),
 ('is', 55),
 ('cancer', 52),
 ('poor', 49),
 ('fluid', 49),
 ('aortic', 49),
 ('stage', 47),
 ('post', 45),
 ('liver', 44),
 (';', 43),
 ('picc', 42),
 ('subdural', 42),
 ('-', 42),
 ('reason_for_this_examination', 41),
 ('made', 41),
 ('a', 41),
 ('herniation', 41),
 ('status', 41),
 ('awaiting', 40),
 ('draining', 39),
 ('prognosis', 39),
 ('extremity', 39)]

In [62]:
set(words) & set(words_tf), set(words) - set(words_tf), set(words_tf) - set(words)

({'ascites',
  'awaiting',
  'cancer',
  'coccyx',
  'dnr',
  'duoderm',
  'meeting',
  'metastatic',
  'picc',
  'prognosis',
  'subdural'},
 {',',
  '-',
  '.',
  '/',
  ':',
  ';',
  'a',
  'and',
  'aortic',
  'care',
  'contraindications',
  'diagnosis',
  'disease',
  'draining',
  'edema',
  'extremity',
  'family',
  'fluid',
  'for',
  'from',
  'herniation',
  'is',
  'line',
  'liver',
  'made',
  'multiple',
  'of',
  'p',
  'placement',
  'pleural',
  'poor',
  'post',
  'reason_for_this_examination',
  'skin',
  'stage',
  'status',
  'team',
  'to',
  'with'},
 {'7.31',
  'admissions',
  'amiodarone',
  'anasarca',
  'ankylosing',
  'bmt',
  'brown',
  'chronically',
  'cmo',
  'coagulopathy',
  'coarse',
  'dark',
  'dialysis',
  'endovascular',
  'enteric',
  'hospice',
  'icteric',
  'iv_contrast',
  'j',
  'jaundiced',
  'listerine',
  'metastases',
  'monday',
  'narrowing',
  'onc',
  'palliative',
  'paracentesis',
  'pea',
  'prednisone',
  'rf',
  'tears',
  'th

Baselines
==========

In [ ]:
lda = LDA({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name})
lda.train(train_data)
lda.evaluate(dev_data, save_results=True)
print(lda.get_topics(n=10))
topics = lda.get_topics(n=10)
print([topics[i] for i in np.argsort(lda.lda_classifier.coef_[0])])

In [ ]:
[topics[i] for i in np.argsort(lda.lda_classifier.coef_[0])]

In [ ]:
preds = outputs['predictions'][:, 0]

In [ ]:
from common import collapse_and_print_word_attn, print_sent_attn

In [ ]:
n = 50
collapse_and_print_word_attn(data.vocab, dev_data.X[n], outputs['word_attentions'][n])
print_sent_attn(data.vocab, dev_data.X[n], outputs['sentence_attentions'][n])

In [ ]:
outputs['sentence_attentions'][0]

In [ ]:
from scipy.stats import spearmanr, kendalltau

In [ ]:
corrs = [kendalltau(range(len(outputs['sentence_attentions'][i])), outputs['sentence_attentions'][i]) 
         for i in range(len(outputs['sentence_attentions']))]

In [ ]:
rho, leng = zip(*[(x[0], y) for x, y in zip(corrs, [len(z) for z in outputs['sentence_attentions']]) if x[0] == x[0]])

In [ ]:
plt.hist(rho, bins=30)

In [ ]:
pval, leng1 = zip(*[(x[1], y) for x, y in zip(corrs, [len(z) for z in outputs['sentence_attentions']]) if x[1] == x[1]])

Saving Models
==============

In [ ]:
from common import generate_latex_tables
keys_to_use = ['1/precision', '1/recall', '1/f1-score', 'accuracy', 'roc_auc', 'pr_auc']
generate_latex_tables(data, keys_to_use)